In [11]:
from qiskit import QuantumCircuit, transpile, execute, IBMQ
from states import Key, Ciphertext
from global_parameters import GlobalParameters
from encryption_circuit import encrypt
from verification_circuit import verify_deletion_counts
from decryption_circuit import create_decryption_circuit, decrypt_results
from deletion_circuit import delete
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeMontreal

In [12]:
# Qiskit parameters
provider = IBMQ.load_account()
noise_model = FakeMontreal()
backend = AerSimulator.from_backend(noise_model)
# backend = provider.get_backend("ibmq_santiago")
optimization_level = 1
shots = 1000

ibmqfactory.load_account:WARNING:2022-06-06 09:49:45,077: Credentials are already in use. The existing account in the session will be replaced.


In [13]:
# Utility function to run a circuit, and return the result counts
def run_and_measure(circuit: QuantumCircuit) -> dict[str, int]:
    print(circuit.draw())
    transpiled_circuit = transpile(
        circuit, backend=backend, optimization_level=0)
    # result = execute(transpiled_circuit, backend=backend,
    #                  optimization_level=optimization_level, shots=1).result()
    result = backend.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    # Reverse the string since the most significant qubit is at the 0th index of the measurement string
    reversed_keys_dict = {}
    for key, value in counts.items():
        reversed_keys_dict[key[::-1]] = value
    return reversed_keys_dict

In [14]:
# Encrypt 
global_params = GlobalParameters(1)
key = Key.generate_key(global_params)
message = "001011"
ciphertext = encrypt(message, key, global_params)
original_circuit = ciphertext.circuit.copy() # We might modify this circuit in subsequent steps

In [15]:
# Test 1 - delete
deletion_circuit = delete(ciphertext)
deletion_counts = run_and_measure(deletion_circuit)
verify_deletion_counts(deletion_counts, key)

         ┌───┐      ░ ┌───┐ ░ ┌─┐                                          
    q_0: ┤ H ├──────░─┤ H ├─░─┤M├──────────────────────────────────────────
         ├───┤      ░ ├───┤ ░ └╥┘┌─┐                                       
    q_1: ┤ X ├──────░─┤ H ├─░──╫─┤M├───────────────────────────────────────
         ├───┤      ░ ├───┤ ░  ║ └╥┘┌─┐                                    
    q_2: ┤ X ├──────░─┤ H ├─░──╫──╫─┤M├────────────────────────────────────
         └───┘      ░ ├───┤ ░  ║  ║ └╥┘┌─┐                                 
    q_3: ───────────░─┤ H ├─░──╫──╫──╫─┤M├─────────────────────────────────
         ┌───┐      ░ ├───┤ ░  ║  ║  ║ └╥┘┌─┐                              
    q_4: ┤ X ├──────░─┤ H ├─░──╫──╫──╫──╫─┤M├──────────────────────────────
         ├───┤      ░ ├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐                           
    q_5: ┤ H ├──────░─┤ H ├─░──╫──╫──╫──╫──╫─┤M├───────────────────────────
         ├───┤┌───┐ ░ ├───┤ ░  ║  ║  ║  ║  ║ └╥┘┌─┐                        
    q_6: ┤ X

In [16]:
# Test 2 - decrypt
decryption_circuit = create_decryption_circuit(key, ciphertext)
decryption_counts = run_and_measure(decryption_circuit)
decrypt_results(decryption_counts, key, ciphertext, message)

         ┌───┐      ░ ┌───┐ ░ ┌─┐                                          
    q_0: ┤ H ├──────░─┤ H ├─░─┤M├──────────────────────────────────────────
         ├───┤      ░ └───┘ ░ └╥┘┌─┐                                       
    q_1: ┤ X ├──────░───────░──╫─┤M├───────────────────────────────────────
         ├───┤      ░       ░  ║ └╥┘┌─┐                                    
    q_2: ┤ X ├──────░───────░──╫──╫─┤M├────────────────────────────────────
         └───┘      ░       ░  ║  ║ └╥┘┌─┐                                 
    q_3: ───────────░───────░──╫──╫──╫─┤M├─────────────────────────────────
         ┌───┐      ░       ░  ║  ║  ║ └╥┘┌─┐                              
    q_4: ┤ X ├──────░───────░──╫──╫──╫──╫─┤M├──────────────────────────────
         ├───┤      ░ ┌───┐ ░  ║  ║  ║  ║ └╥┘┌─┐                           
    q_5: ┤ H ├──────░─┤ H ├─░──╫──╫──╫──╫──╫─┤M├───────────────────────────
         ├───┤┌───┐ ░ ├───┤ ░  ║  ║  ║  ║  ║ └╥┘┌─┐                        
    q_6: ┤ X

In [17]:
# Test 3 - delete and then decrypt
deletion_circuit = delete(ciphertext)
ciphertext.circuit = deletion_circuit # To use the new circuit as the starting point of the decryption circuit
decryption_circuit = create_decryption_circuit(key, ciphertext)
counts = run_and_measure(decryption_circuit)
print(counts)
deletion_counts = {}
decryption_counts = {}
for measurement, count in counts.items():
    # Qiskit will return a space-separated string of the two measurements. 
    # run_and_measure will reverse the string so that the first substring is the first measurement.
    deletion, decryption = measurement.split(" ")
    deletion_counts[deletion] = deletion_counts.get(deletion, 0) + 1
    decryption_counts[decryption] = decryption_counts.get(decryption, 0) + 1
verify_deletion_counts(deletion_counts, key)
decrypt_results(decryption_counts, key, ciphertext, message)

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

          ┌───┐      ░ ┌───┐ ░ ┌─┐                                           ░ »
     q_0: ┤ H ├──────░─┤ H ├─░─┤M├───────────────────────────────────────────░─»
          ├───┤      ░ ├───┤ ░ └╥┘┌─┐                                        ░ »
     q_1: ┤ X ├──────░─┤ H ├─░──╫─┤M├────────────────────────────────────────░─»
          ├───┤      ░ ├───┤ ░  ║ └╥┘┌─┐                                     ░ »
     q_2: ┤ X ├──────░─┤ H ├─░──╫──╫─┤M├─────────────────────────────────────░─»
          └───┘      ░ ├───┤ ░  ║  ║ └╥┘┌─┐                                  ░ »
     q_3: ───────────░─┤ H ├─░──╫──╫──╫─┤M├──────────────────────────────────░─»
          ┌───┐      ░ ├───┤ ░  ║  ║  ║ └╥┘┌─┐                               ░ »
     q_4: ┤ X ├──────░─┤ H ├─░──╫──╫──╫──╫─┤M├───────────────────────────────░─»
          ├───┤      ░ ├───┤ ░  ║  ║  ║  ║ └╥┘┌─┐                            ░ »
     q_5: ┤ H ├──────░─┤ H ├─░──╫──╫──╫──╫──╫─┤M├────────────────────────────░─»
          ├───┤┌───┐ ░ ├───┤